In [ ]:
from kubernetes import client, config
import yaml

# Load in-cluster credentials if running in Kubeflow
config.load_incluster_config()

# Variables for the task are defined here
namespace = "vllm"
task_name = "deploy-model"
model_url = "<REPLACE_ME_WITH_QUAY_ROUTE>/admin/granite-3.3-2b-instruct:latest"

# Example TaskRun spec (you can load this from YAML instead)
taskrun_yaml = f"""
apiVersion: tekton.dev/v1
kind: TaskRun
metadata:
  generateName: {task_name}-run-
  namespace: {namespace}
  labels:
    app.kubernetes.io/managed-by: tekton-pipelines
    tekton.dev/task: {task_name}
spec:
  serviceAccountName: pipeline
  taskRef:
    kind: Task
    name: {task_name}
  params:
    - name: releaseName
      value: "granite-2b"
    - name: namespace
      value: vllm
    - name: chartUrl
      value: "https://redhat-ai-services.github.io/helm-charts/"
    - name: valuesContent
      value: |
          deploymentMode: RawDeployment
          fullnameOverride: "granite-2b"
          model:
            modelNameOverride: "granite-2b"
            uri: "oci://{model_url}"
            args:
              - "--disable-log-requests"
              - "--max-num-seqs=32"
  timeout: 1h0m0s
  workspaces:
    - name: shared-workspace
      persistentVolumeClaim:
        claimName: guidellm-output-pvc
"""

taskrun_dict = yaml.safe_load(taskrun_yaml)

api = client.CustomObjectsApi()
resp = api.create_namespaced_custom_object(
    group="tekton.dev",
    version="v1",
    namespace=namespace,
    plural="taskruns",
    body=taskrun_dict
)

print(f"✅ Created TaskRun: {resp['metadata']['name']}")

✅ Created TaskRun: deploy-model-run-hsflw


In [4]:
import time

name = resp['metadata']['name']
while True:
    run = api.get_namespaced_custom_object(
        group="tekton.dev",
        version="v1",
        namespace=namespace,
        plural="taskruns",
        name=name
    )
    conds = run.get("status", {}).get("conditions", [])
    if conds:
        status = conds[0].get("status")
        reason = conds[0].get("reason")
        print(f"TaskRun status: {reason}")
        if status == "True":
            print("✅ TaskRun succeeded")
            break
        elif status == "False":
            raise RuntimeError(f"❌ TaskRun failed: {reason}")
    time.sleep(5)


TaskRun status: Succeeded
✅ TaskRun succeeded
